In [1]:
import os
import os.path as osp
import shutil
import glob

import torch
from torch_geometric.data import InMemoryDataset, download_url, extract_zip
from torch_geometric.io import read_off

def download(root='data/modelnet/', name='40'):
    urls = {
        '10':
        'http://vision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip',
        '40': 'http://modelnet.cs.princeton.edu/ModelNet40.zip'
    }
    os.makedirs(root, exist_ok=True)
    path = download_url(urls[name], root)
    extract_zip(path, root)
    os.unlink(path)
    folder = osp.join(root, 'ModelNet{}'.format(self.name))
    shutil.rmtree(self.raw_dir)
    os.rename(folder, self.raw_dir)

    # Delete osx metadata generated during compression of ModelNet10
    metadata_folder = osp.join(root, '__MACOSX')
    if osp.exists(metadata_folder):
        shutil.rmtree(metadata_folder)

In [ ]:
#download()

Extracting data/modelnet/ModelNet40.zip


In [28]:
import numpy as np
import h5py
from tqdm.notebook import tqdm
import os
def extract(path):
    f = open(path, 'r')
    lines = f.readlines()
    if lines[0] == 'OFF\n':
        num = int(float(lines[1].split(" ")[0]))
        pts = []
        for i in range(2, 2+num):
            temp = lines[i][:-1].split(' ')
            pts.append([float(temp[0]), float(temp[1]), float(temp[2])])
    else:
        num = int(float(lines[0].split(" ")[0][3:]))
        pts = []
        for i in range(1, 1+num):
            temp = lines[i][:-1].split(' ')
            pts.append([float(temp[0]), float(temp[1]), float(temp[2])])
    return pts

def create_pt(base = 'data/modelnet/ModelNet40/', output_base = 'data/modelnet/modelnet_points/'):
    paths = {}
    for obj in os.listdir(base):
        if obj != '.DS_Store' and obj != 'README.txt':
            obj_base_train = base + obj + '/train/'
            obj_base_test = base + obj + '/test/'
            paths[obj] = []
            for sample in os.listdir(obj_base_train):
                paths[obj].append(obj_base_train + sample)
            for sample in os.listdir(obj_base_test):
                paths[obj].append(obj_base_test + sample)
                
    os.makedirs(output_base, exist_ok=True)

    for obj in tqdm(paths.keys()):
        h5f = h5py.File(output_base + str(obj)+'.h5', 'w')
        for path, i in zip(paths[obj], range(len(paths[obj]))):
            if path[-3:] == 'off':
                temp = np.array(extract(path))
                h5f.create_dataset('object_' + str(i), data=temp)
        h5f.close()

In [30]:
create_pt()

  0%|          | 0/40 [00:00<?, ?it/s]

In [11]:
from src import graph_construct
import os
import numpy as np
import h5py

def construct_graph_knn(pts_num = 1000, base = 'data/modelnet/modelnet_points/', k=15):
    output_base = 'data/modelnet/modelnet_graph_k{x}'.format(x=k)
    os.makedirs(output_base, exist_ok=True)
    for obj in os.listdir(base):
        cat = obj[:-3]
        print(obj)
        if obj[-2:] == 'h5':
            os.makedirs(output_base + '/' + cat, exist_ok=True)
            f = h5py.File(base + obj, 'r')
            for key in f.keys():
                if f[key][:].shape[0] >= pts_num:
                    pts = graph_construct.pts_norm(graph_construct.pts_sample(f[key][:], pts_num))
                    if np.isnan(pts).any():
                        continue
                    temp = graph_construct.graph_construct_kneigh(pts, k=k)
                    filename = output_base + '/' + cat + '/' + key + '.h5'
                    print(filename)
                    out = h5py.File(filename, 'w')
                    out.create_dataset('edges', data=temp[0])
                    out.create_dataset('edge_weight', data=temp[1])
                    out.create_dataset('nodes', data=pts)
                    out.close()
    print('After processing, numbers of samples for each category are:')
    for obj in os.listdir(output_base):
        print(obj, len(os.listdir(output_base + '/' + obj)))

In [12]:
def construct_graph_radius(pts_num = 1000, base = 'data/modelnet/modelnet_points/', radius=0.1):
    output_base = 'data/modelnet/modelnet_graph_r{x}/'.format(x=radius)
    os.makedirs(output_base, exist_ok=True)
    for obj in os.listdir(base):
        cat = obj[:-3]
        print(obj)
        if obj[-2:] == 'h5':
            os.makedirs(output_base + '/' + cat, exist_ok=True)
            f = h5py.File(base + obj, 'r')
            for key in f.keys():
                if f[key][:].shape[0] >= pts_num:
                    pts = graph_construct.pts_norm(graph_construct.pts_sample(f[key][:], pts_num))
                    if np.isnan(pts).any():
                        continue
                    temp = graph_construct.graph_construct_radius(pts, r=radius)
                    filename = output_base + '/' + cat + '/' + key + '.h5'
                    print(filename)
                    out = h5py.File(filename, 'w')
                    out.create_dataset('edges', data=temp[0])
                    out.create_dataset('edge_weight', data=temp[1])
                    out.create_dataset('nodes', data=pts)
                    out.close()
    print('After processing, numbers of samples for each category are:')
    for obj in os.listdir(output_base):
        print(obj, len(os.listdir(output_base + '/' + obj)))

In [ ]:
construct_graph_radius(radius=0.05)

xbox.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_0.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_1.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_10.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_100.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_101.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_103.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_104.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_105.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_106.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_107.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_109.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_11.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_110.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_111.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_113.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_114.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_115.h5
data/modelnet/modelnet_graph_r0.05//xbox/object_116.h5
data/mod

data/modelnet/modelnet_graph_r0.05//chair/object_149.h5
data/modelnet/modelnet_graph_r0.05//chair/object_15.h5
data/modelnet/modelnet_graph_r0.05//chair/object_150.h5
data/modelnet/modelnet_graph_r0.05//chair/object_151.h5
data/modelnet/modelnet_graph_r0.05//chair/object_152.h5
data/modelnet/modelnet_graph_r0.05//chair/object_153.h5
data/modelnet/modelnet_graph_r0.05//chair/object_154.h5
data/modelnet/modelnet_graph_r0.05//chair/object_156.h5
data/modelnet/modelnet_graph_r0.05//chair/object_157.h5
data/modelnet/modelnet_graph_r0.05//chair/object_158.h5
data/modelnet/modelnet_graph_r0.05//chair/object_159.h5
data/modelnet/modelnet_graph_r0.05//chair/object_160.h5
data/modelnet/modelnet_graph_r0.05//chair/object_161.h5
data/modelnet/modelnet_graph_r0.05//chair/object_162.h5
data/modelnet/modelnet_graph_r0.05//chair/object_163.h5
data/modelnet/modelnet_graph_r0.05//chair/object_164.h5
data/modelnet/modelnet_graph_r0.05//chair/object_165.h5
data/modelnet/modelnet_graph_r0.05//chair/object_

data/modelnet/modelnet_graph_r0.05//chair/object_305.h5
data/modelnet/modelnet_graph_r0.05//chair/object_306.h5
data/modelnet/modelnet_graph_r0.05//chair/object_307.h5
data/modelnet/modelnet_graph_r0.05//chair/object_308.h5
data/modelnet/modelnet_graph_r0.05//chair/object_309.h5
data/modelnet/modelnet_graph_r0.05//chair/object_310.h5
data/modelnet/modelnet_graph_r0.05//chair/object_311.h5
data/modelnet/modelnet_graph_r0.05//chair/object_313.h5
data/modelnet/modelnet_graph_r0.05//chair/object_314.h5
data/modelnet/modelnet_graph_r0.05//chair/object_315.h5
data/modelnet/modelnet_graph_r0.05//chair/object_316.h5
data/modelnet/modelnet_graph_r0.05//chair/object_317.h5
data/modelnet/modelnet_graph_r0.05//chair/object_318.h5
data/modelnet/modelnet_graph_r0.05//chair/object_319.h5
data/modelnet/modelnet_graph_r0.05//chair/object_32.h5
data/modelnet/modelnet_graph_r0.05//chair/object_320.h5
data/modelnet/modelnet_graph_r0.05//chair/object_321.h5
data/modelnet/modelnet_graph_r0.05//chair/object_

data/modelnet/modelnet_graph_r0.05//chair/object_462.h5
data/modelnet/modelnet_graph_r0.05//chair/object_463.h5
data/modelnet/modelnet_graph_r0.05//chair/object_464.h5
data/modelnet/modelnet_graph_r0.05//chair/object_465.h5
data/modelnet/modelnet_graph_r0.05//chair/object_466.h5
data/modelnet/modelnet_graph_r0.05//chair/object_467.h5
data/modelnet/modelnet_graph_r0.05//chair/object_468.h5
data/modelnet/modelnet_graph_r0.05//chair/object_469.h5
data/modelnet/modelnet_graph_r0.05//chair/object_47.h5
data/modelnet/modelnet_graph_r0.05//chair/object_470.h5
data/modelnet/modelnet_graph_r0.05//chair/object_471.h5
data/modelnet/modelnet_graph_r0.05//chair/object_472.h5
data/modelnet/modelnet_graph_r0.05//chair/object_473.h5
data/modelnet/modelnet_graph_r0.05//chair/object_474.h5
data/modelnet/modelnet_graph_r0.05//chair/object_475.h5
data/modelnet/modelnet_graph_r0.05//chair/object_476.h5
data/modelnet/modelnet_graph_r0.05//chair/object_477.h5
data/modelnet/modelnet_graph_r0.05//chair/object_

data/modelnet/modelnet_graph_r0.05//chair/object_613.h5
data/modelnet/modelnet_graph_r0.05//chair/object_614.h5
data/modelnet/modelnet_graph_r0.05//chair/object_615.h5
data/modelnet/modelnet_graph_r0.05//chair/object_617.h5
data/modelnet/modelnet_graph_r0.05//chair/object_618.h5
data/modelnet/modelnet_graph_r0.05//chair/object_619.h5
data/modelnet/modelnet_graph_r0.05//chair/object_62.h5
data/modelnet/modelnet_graph_r0.05//chair/object_620.h5
data/modelnet/modelnet_graph_r0.05//chair/object_621.h5
data/modelnet/modelnet_graph_r0.05//chair/object_622.h5
data/modelnet/modelnet_graph_r0.05//chair/object_623.h5
data/modelnet/modelnet_graph_r0.05//chair/object_624.h5
data/modelnet/modelnet_graph_r0.05//chair/object_625.h5
data/modelnet/modelnet_graph_r0.05//chair/object_626.h5
data/modelnet/modelnet_graph_r0.05//chair/object_627.h5
data/modelnet/modelnet_graph_r0.05//chair/object_628.h5
data/modelnet/modelnet_graph_r0.05//chair/object_629.h5
data/modelnet/modelnet_graph_r0.05//chair/object_

...

data/modelnet/modelnet_graph_r0.05//monitor/object_352.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_353.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_354.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_355.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_356.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_358.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_36.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_360.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_361.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_363.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_364.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_365.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_367.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_369.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_37.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_370.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_371.h5
data/modelnet/mo

data/modelnet/modelnet_graph_r0.05//monitor/object_63.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_65.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_67.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_69.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_7.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_70.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_72.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_73.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_74.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_75.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_76.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_77.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_78.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_79.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_80.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_84.h5
data/modelnet/modelnet_graph_r0.05//monitor/object_85.h5
data/modelnet/modelnet_graph_r0.

data/modelnet/modelnet_graph_r0.05//mantel/object_209.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_21.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_210.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_211.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_212.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_213.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_214.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_215.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_216.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_217.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_218.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_219.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_22.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_222.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_223.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_224.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_225.h5
data/modelnet/modelnet_graph_r0.0

data/modelnet/modelnet_graph_r0.05//mantel/object_35.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_350.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_351.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_352.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_353.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_355.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_356.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_357.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_358.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_359.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_360.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_361.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_362.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_363.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_364.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_365.h5
data/modelnet/modelnet_graph_r0.05//mantel/object_366.h5
data/modelnet/modelnet_graph_r0.

data/modelnet/modelnet_graph_r0.05//tv_stand/object_183.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_184.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_188.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_19.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_190.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_193.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_194.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_198.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_199.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_2.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_20.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_200.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_204.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_212.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_214.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_216.h5
data/modelnet/modelnet_graph_r0.05//tv_stand/object_218.h5
d

In [ ]:
construct_graph_radius(radius=0.15)

In [ ]:
construct_graph_radius(radius=0.2)

In [ ]:
construct_graph_knn(k=20)

In [ ]:
construct_graph_knn(k=25)